## 文本预处理
### 常见步骤
1. 将文本作为字符串加载到内存中
2. 将字符串拆分为词元
3. 建立一个词表，将拆分的词元映射到数字索引
4. 将文本转换为数字索引序列，方便模型操作

In [1]:
import collections
import re
from d2l import torch as d2l

- 读取数据集

In [2]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():
    with (open(d2l.download('time_machine'), 'r')) as f:
        lines = f.readlines()
    # 把每一行中的所有非字母字符删除，并全部转为小写
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]
lines = read_time_machine()
print(f'#文本总行数：{len(lines)}')
print(lines[0])
print(lines[10])

#文本总行数：3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


### 词元化

In [5]:
def tokenize(lines, token='word'):
    """将输入的文本转换成单词或者字符词元，返回一个list of list"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        raise TypeError(f"未知词元类型：{token}")
tokens = tokenize(lines, 'word')
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


### 词表
- 上述得到的token的类型是一个list of list，每一个list中是一个个token（单词或者字符）,但是模型的输入应该是一个个数字，所以我们建立一个词表，建立从单词或者字符到数字的映射
- **语料（corpus）：**对唯一词元进行统计的结果
- 根据每个唯一token出现的唯一频率，为其分配一个数字索引

In [ ]:
class Vocab:
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        counter = count_corpus(tokens)
        # 对字典中的对从小到大排序
        self._token_freqs = sorted(counter.items(), key=lambda x:x[1], reverse=True)
        # 未知token索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens # 除了语料库中的token仍然需要加上的特殊token
        self.token_to_idx = {token:idx for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            else:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        """此getitem函数是接收token返回idx"""
        if not isinstance(tokens, (tuple, list)): # 对于单个token, 返回其索引
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens] # 对于多个token，分解列表，递归查询，返回idx
    
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.to_tokens[index] for index in indices]

    @property
    def unk(self):
        return 0
    
    @property
    def token_freq(self):
        return self._token_freqs


# 统计词元频率的函数
def count_corpus(tokens):
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)
        

In [7]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [9]:
for i in [0, 10]:
    print("文本：", tokens[i])
    print('索引:', vocab[tokens[i]])

文本： ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本： ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [14]:
def load_corpus_machine(max_tokens=-1):
    """返回time_machine的corpus和Vocab"""
    lines = read_time_machine()
    tokens = tokenize(lines, token='char')
    vocab = Vocab(tokens)
    # 因为time_machine数据每个文本不一定是一个句子或者一个段落
    # 所以将所有文本展平放到一个列表中
    # corpus得到的是所有字母的索引，存在了一个列表中
    corpus = [vocab[token] for line in lines for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_machine()
len(corpus), len(vocab)

(170580, 28)

- 证明这个txt文件中共有170580个token，其中Vocab中有28个，包括26个英文字母以及<unk>和空格